# Introduction to Data Science - miniproject
## Preprocessing of data

## Helsinki districts to a clean data frame and csv

In [10]:
import pandas as pd
import numpy as np

# Install required version of openpyxl if needed
#!pip install openpyxl==3.0.10

# Read excel file from repository, remove the introductory row 0 and the last row without real data
# Original web address to file: 'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Helsingin_aluejaot_2016.xlsx'
districts = pd.read_excel('../Data_sourcefiles/Helsingin_aluejaot_2016.xlsx', 
                          sheet_name='Aluejako', skiprows=1, skipfooter=1)

districts

,KOKOTUN,KUNTA,SUUR,PERUS,OSA,PIEN,Nimi,Helsingin tilastoaluetunnus,Tilastokeskuksen aluetunnus
0,910000000,91,0,0,0,0,0,091 Helsinki,091 Helsinki
1,911000000,91,1,0,0,0,Eteläinen,091 1 Eteläinen suurpiiri,0911 1. Eteläinen suurpiiri
2,911101000,91,1,101,0,0,Vironniemi,091 101 Vironniemen peruspiiri,09111 101 Vironniemen peruspiiri
3,911101010,91,1,101,10,0,Kruununhaka,091 10 Kruununhaka,09111010 010 Kruununhaka
4,9111010101,91,1,101,10,1,NaN,1,091110101 0101
...,...,...,...,...,...,...,...,...,...
590,9188015800,91,8,801,580,0,NaN,0,091815800 5800
591,918801591,91,8,801,591,0,Landbo,091 591 Landbo,09181591 591 Landbo
592,9188015910,91,8,801,591,0,NaN,0,091815910 5910
593,918801592,91,8,801,592,0,Puroniitty,091 592 Puroniitty,09181592 592 Puroniitty


In [11]:
# Remove unneeded columns
districts = districts.drop(['KOKOTUN', 'PIEN'], axis=1)

# Change numerical columns to ints 
districts = districts.astype({'KUNTA': int,
                        'SUUR': int,
                        'PERUS': int,
                        'OSA': int,
                       })

# Remove level for 'pienalue'
districts = districts[districts['Nimi'].notna()]

# Filter out the row for city level 
districts = districts[districts['SUUR'] != 0]

# Make a new column for the names of 'suurpiiri' and fill values down
districts['Suurpiiri'] = districts['Nimi'].where((districts['SUUR'] != 0) & (districts['PERUS'] == 0)).ffill()

# Make a new column for the names of 'peruspiiri' and fill values down
districts['Peruspiiri'] = districts['Nimi'].where((districts['PERUS'] != 0) & (districts['OSA'] == 0)).ffill()
# Replace name with 0 for level 'suurpiiri'
districts['Peruspiiri'] = districts['PERUS'].where(districts['PERUS'] == 0, districts['Peruspiiri'])


# Make a new column for the names of 'osa-alue' and fill values down
districts['Osa-alue'] = districts['Nimi'].where(districts['OSA'] != 0).ffill()
# Replace name with 0 for levels 'suurpiiri' and 'peruspiiri',
districts['Osa-alue'] = districts['OSA'].where(districts['OSA'] == 0, districts['Osa-alue'])


# Rename columns with English names
districts.columns = ['Municipality', 'MAJOR', 'BASIC', 'SUB', 'Name', 'Helsinki statistical area code', 
                     'Statistics Finland area code', 'Major district', 'District', 'Sub-district']

# # Remove original rows for 'major district' (suurpiiri) and 'district' (peruspiiri) if more flat data is needed
# districts = districts[districts['SUB'] != 0]

districts

,Municipality,MAJOR,BASIC,SUB,Name,Helsinki statistical area code,Statistics Finland area code,Major district,District,Sub-district
1,91,1,0,0,Eteläinen,091 1 Eteläinen suurpiiri,0911 1. Eteläinen suurpiiri,Eteläinen,0,0
2,91,1,101,0,Vironniemi,091 101 Vironniemen peruspiiri,09111 101 Vironniemen peruspiiri,Eteläinen,Vironniemi,0
3,91,1,101,10,Kruununhaka,091 10 Kruununhaka,09111010 010 Kruununhaka,Eteläinen,Vironniemi,Kruununhaka
8,91,1,101,20,Kluuvi,091 20 Kluuvi,09111020 020 Kluuvi,Eteläinen,Vironniemi,Kluuvi
16,91,1,101,80,Katajanokka,091 80 Katajanokka,09111080 080 Katajanokka,Eteläinen,Vironniemi,Katajanokka
...,...,...,...,...,...,...,...,...,...,...
585,91,8,801,560,Salmenkallio,091 560 Salmenkallio,09181560 560 Salmenkallio,Östersundom,Östersundom,Salmenkallio
587,91,8,801,570,Talosaari,091 570 Talosaari,09181570 570 Talosaari,Östersundom,Östersundom,Talosaari
589,91,8,801,580,Karhusaari,091 580 Karhusaari,09181580 580 Karhusaari,Östersundom,Östersundom,Karhusaari
591,91,8,801,591,Landbo,091 591 Landbo,09181591 591 Landbo,Östersundom,Östersundom,Landbo


In [12]:
# Write districts dataframe to csv file in data folder
districts.to_csv('../data/Helsinki_districts.csv', index=False)

## One-person households data

In [13]:
import pandas as pd
import numpy as np

# Install required version of openpyxl if needed
#!pip install openpyxl==3.0.10

# Read excel file from repository, remove the introductory rows 0-3 and the last row without real data
# Original web address to file: 'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Vaestotilastot/2021/Hki_vaesto_taulu17c.xlsx'
# File downloadable at https://www.avoindata.fi/data/fi/dataset/yksinasuvat-yhden-hengen-asuntokunnat-helsingissukupuolen-ja-imukaan-peruspiireitt-2004-alkaen
'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Helsingin_aluejaot_2016.xlsx'
ones = pd.read_excel('../Data_sourcefiles/Hki_vaesto_taulu17c_yksinasuvat.xlsx', 
                          sheet_name='2022', skiprows=4, skipfooter=1)

ones

,Sukupuoli,Yhteensä,-19,20-29,30-39,40-49,50-59,60-64,65-69,70-74,75-79,80-84,85-
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Koko Helsinki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yhteensä,175045.0,2266.0,36455.0,29909.0,19363.0,23806.0,12679.0,12200.0,12212.0,10130.0,7711.0,8314.0
3,Miehet,74602.0,774.0,16254.0,16608.0,11534.0,11258.0,4956.0,4094.0,3450.0,2511.0,1576.0,1587.0
4,Naiset,100443.0,1492.0,20201.0,13301.0,7829.0,12548.0,7723.0,8106.0,8762.0,7619.0,6135.0,6727.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,801 Östersundom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,Yhteensä,113.0,1.0,4.0,9.0,17.0,26.0,12.0,9.0,9.0,11.0,8.0,7.0
213,Miehet,68.0,1.0,2.0,7.0,11.0,16.0,7.0,6.0,5.0,7.0,3.0,3.0
214,Naiset,45.0,0.0,2.0,2.0,6.0,10.0,5.0,3.0,4.0,4.0,5.0,4.0


In [14]:
# Change column 'sukupuoli' to type string
ones = ones.astype({'Sukupuoli': str})

# Remove trailing spaces in column 'Sukupuoli'
ones['Sukupuoli'] = ones['Sukupuoli'].apply(lambda x: x.strip())


# Remove 'total' rows ('yhteensä')
ones = ones[ones['Sukupuoli'] != 'Yhteensä']

# Add new column for 'suurpiiri'
ones['Suurpiiri'] = ones['Sukupuoli'].where(ones['Sukupuoli'].str.contains('suurpiiri')).ffill()

# Add new column for 'peruspiiri'
ones['Peruspiiri'] = ones['Sukupuoli'].where((ones['Sukupuoli'] != 'Miehet') & (ones['Sukupuoli'] != 'Naiset')).ffill()

# Remove rows with empty values
ones = ones.dropna()

# Remove rows for level 'suurpiiri'
ones = ones[~ones['Peruspiiri'].str.contains('suurpiiri')]

# Change numerical columns to type int
ones = ones.astype({'Yhteensä': int,
                    -19: int,
                    '20-29': int,
                    '30-39': int, 
                    '50-59': int,
                    '60-64': int,
                    '65-69': int,
                    '70-74': int,
                    '75-79': int,
                    '80-84': int,
                    '85-': int})

# Translate 'sukupuoli' values to English
ones['Sukupuoli'] = ones['Sukupuoli'].replace({'Miehet': 'Men', 'Naiset': 'Women'})

# Rename columns with English names
ones.columns = ['Gender', 'All ages', 'Age -19', 'Age 20-29', 'Age 30-39', 'Age 40-49', 'Age 50-59', 'Age 60-64', 'Age 65-69', 'Age 70-74', 'Age 75-79', 'Age 80-84', 'Age 85', 'Major district', 'District']
# Add suffix to column names to point them to one-person households
ones = ones.add_suffix(', one-person hh')

ones


,"Gender, one-person hh","All ages, one-person hh","Age -19, one-person hh","Age 20-29, one-person hh","Age 30-39, one-person hh","Age 40-49, one-person hh","Age 50-59, one-person hh","Age 60-64, one-person hh","Age 65-69, one-person hh","Age 70-74, one-person hh","Age 75-79, one-person hh","Age 80-84, one-person hh","Age 85, one-person hh","Major district, one-person hh","District, one-person hh"
13,Men,1306,18,293,248,210.0,195,80,67,76,53,43,23,1 Eteläinen suurpiiri,101 Vironniemi
14,Women,1925,31,428,230,161.0,213,106,159,174,168,135,120,1 Eteläinen suurpiiri,101 Vironniemi
18,Men,3117,28,847,647,517.0,427,191,132,109,84,70,65,1 Eteläinen suurpiiri,102 Ullanlinna
19,Women,4575,92,1254,759,379.0,481,278,298,314,310,210,200,1 Eteläinen suurpiiri,102 Ullanlinna
23,Men,5299,77,1598,1238,795.0,661,278,178,162,162,69,81,1 Eteläinen suurpiiri,103 Kampinmalmi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Women,5505,86,623,421,358.0,856,594,571,562,519,452,463,7 Itäinen suurpiiri,703 Mellunkylä
203,Men,3968,38,592,754,617.0,797,348,254,188,159,113,108,7 Itäinen suurpiiri,704 Vuosaari
204,Women,5606,49,556,439,426.0,986,544,557,617,518,467,447,7 Itäinen suurpiiri,704 Vuosaari
213,Men,68,1,2,7,11.0,16,7,6,5,7,3,3,8 Östersundomin suurpiiri,801 Östersundom


In [15]:
# Write one-person households dataframe to csv file in data folder
ones.to_csv('../data/Helsinki_oneperson_households.csv', index=False)